In [818]:
import random
import pandas as pd

In [819]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    if node.question.moreless(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

class Decision_Node:
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1
    for col in range(n_features): 
        values = set([row[col] for row in rows])
        for val in values: 
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question

def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def moreless(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val <= self.value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.moreless(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return
    print(spacing + str(node.question))
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [820]:
def createvalues():
    values=[]
    template=[{"label":"Klassrum","minheat":23,"maxheat":26,"minhum":27,"maxhum":37,"minco2":600,"maxco2":1200,"minlux":150,"maxlux":250},{"label":"Fikarum","minheat":25,"maxheat":28,"minhum":37,"maxhum":44,"minco2":0,"maxco2":0,"minlux":300,"maxlux":550}]
    for roomtype in template:
        amount = random.randint(5,11)
        for i in range(0,amount):
            newtemp=random.uniform(roomtype["minheat"], roomtype["maxheat"])
            newhum=random.uniform(roomtype["minhum"], roomtype["maxhum"])
            newco2=random.uniform(roomtype["minco2"], roomtype["maxco2"])
            newlux=random.uniform(roomtype["minlux"], roomtype["maxlux"])
            values.append([newtemp, newhum, newco2, newlux, roomtype["label"]])
    return values

In [821]:
newvalues = pd.DataFrame(createvalues())
print(newvalues)

0          1           2           3         4
0   24.136454  34.973573  798.198831  248.764828  Klassrum
1   25.760936  28.837771  922.656584  162.208856  Klassrum
2   24.559787  31.705634  684.804805  157.609171  Klassrum
3   25.785052  34.727809  994.240861  191.922076  Klassrum
4   23.062577  36.594448  631.630793  244.155565  Klassrum
5   23.437294  31.448671  743.086576  207.874248  Klassrum
6   25.529694  41.725862    0.000000  477.590410   Fikarum
7   26.794609  38.910148    0.000000  471.510825   Fikarum
8   27.060046  39.816394    0.000000  314.333250   Fikarum
9   25.883582  37.587427    0.000000  531.842824   Fikarum
10  26.479317  41.984553    0.000000  320.080933   Fikarum
11  26.826487  41.052601    0.000000  487.608772   Fikarum


In [822]:
import requests

urls = ["https://car9o7qv7j.execute-api.us-east-1.amazonaws.com/iot/device?MAC=84:F3:EB:B4:6F:61","https://nkclsbr32f.execute-api.us-east-1.amazonaws.com/beta/device?MAC=DC:4F:22:5F:43:75"]
Simon = pd.DataFrame()
BBB = pd.DataFrame()

info = requests.get(urls[0]).json()
for k in info['data']:            
    if k['CO2'] < 5000:
        k["time"]="Klassrum"
        Simon = Simon.append(k, ignore_index=True)

info = requests.get(urls[1]).json()
for k in info['data']:
    if k['CO2'] < 5000:
        k["time"]="Fikarum"
        Simon = Simon.append(k, ignore_index=True)

newvalues = Simon
newvalues.columns  = ["CO2", "Hum", "LDR", "Temp", "Rum"]
testing_data = createvalues()
print(newvalues)

CO2   Hum    LDR  Temp       Rum
0   1016.0  31.0  204.0  24.0  Klassrum
1   1078.0  29.0  197.0  25.0  Klassrum
2   1065.0  28.0  209.0  25.0  Klassrum
3    994.0  28.0  219.0  26.0  Klassrum
4    938.0  27.0  211.0  25.0  Klassrum
5    996.0  28.0  212.0  26.0  Klassrum
6    957.0  28.0  198.0  26.0  Klassrum
7    795.0  28.0  195.0  26.0  Klassrum
8    647.0  28.0  182.0  26.0  Klassrum
9   1042.0  29.0  190.0  25.0  Klassrum
10  1116.0  30.0  172.0  26.0  Klassrum
11   821.0  31.0  175.0  26.0  Klassrum
12   894.0  37.0  190.0  23.0  Klassrum
13   868.0  37.0  219.0  24.0  Klassrum
14   839.0  37.0  193.0  24.0  Klassrum
15   862.0  36.0  190.0  24.0  Klassrum
16   826.0  36.0  204.0  24.0  Klassrum
17   824.0  36.0  216.0  24.0  Klassrum
18   575.0  36.0  213.0  24.0  Klassrum
19   603.0  35.0  224.0  24.0  Klassrum
20   759.0  35.0  223.0  24.0  Klassrum
21   778.0  35.0  213.0  24.0  Klassrum
22     0.0  38.0  303.0  26.0   Fikarum
23     0.0  38.0  489.0  25.0   Fikarum
24     

In [823]:
def splitdata(inputdata, n_trees, ownvalue):
    subsets=[]
    splitone=inputdata.sample(frac=0.5, random_state=200)
    splittwo=inputdata.drop(splitone.index)
    if n_trees != 1:
        feedback=splitdata(splitone, n_trees-1, ownvalue+"a")
        feedback2=splitdata(splittwo, n_trees-1, ownvalue+"b")
        for i in range(0,len(feedback)):
            subsets.append(feedback[i])
            subsets.append(feedback2[i])
    else:
        return [splitone,splittwo]
    return subsets

In [824]:
subsets=splitdata(newvalues,3,"a") # fler än 1
print("antalsubsets", len(subsets))
print("antalvärden", len(subsets[0]))
print(subsets[0])

antalsubsets 8
antalvärden 6
       CO2   Hum    LDR  Temp       Rum
0   1016.0  31.0  204.0  24.0  Klassrum
35     0.0  39.0  496.0  27.0   Fikarum
36     0.0  39.0  475.0  27.0   Fikarum
44     0.0  43.0  438.0  26.0   Fikarum
51     0.0  43.0  450.0  26.0   Fikarum
20   759.0  35.0  223.0  24.0  Klassrum


In [825]:
forrest=[]
i=1
header = ["CO2", "Hum", "LDR", "Temp", "Rum"]

for sets in subsets:
    testing_data = sets.tail(5).values.tolist() # no work
    lists = sets.values.tolist()
    print("Tree",i)
    i+=1
    tree = build_tree(lists)
    forrest.append(tree)
    print_tree(tree)

Tree 1
Is Temp >= 26.0?
--> True:
  Predict {'Fikarum': 4}
--> False:
  Predict {'Klassrum': 2}
Tree 2
Is LDR >= 449.0?
--> True:
  Predict {'Fikarum': 5}
--> False:
  Predict {'Klassrum': 2}
Tree 3
Is LDR >= 442.0?
--> True:
  Predict {'Fikarum': 1}
--> False:
  Predict {'Klassrum': 5}
Tree 4
Is Temp >= 25.0?
--> True:
  Predict {'Fikarum': 2}
--> False:
  Predict {'Klassrum': 4}
Tree 5
Is LDR >= 432.0?
--> True:
  Predict {'Fikarum': 5}
--> False:
  Predict {'Klassrum': 2}
Tree 6
Is LDR >= 430.0?
--> True:
  Predict {'Fikarum': 4}
--> False:
  Predict {'Klassrum': 3}
Tree 7
Is LDR >= 441.0?
--> True:
  Predict {'Fikarum': 6}
--> False:
  Predict {'Klassrum': 1}
Tree 8
Is Temp >= 26.0?
--> True:
  Predict {'Fikarum': 4}
--> False:
  Predict {'Klassrum': 3}


In [826]:
def predictioncounter(row,trees,labels):
    length=len(labels)
    predictions=[0 for i in range(length)]
    for tree in trees:
        prediction=classify(row,tree)
        for i in range(length):
            if(prediction.get(labels[i]) is not None):
                predictions[i]+=1
        print("Actual: %s. Predicted: %s" %(row[-1], print_leaf(classify(row, tree))))
    return predictions

In [827]:
def randomforest(trees, data):
    labels=["Fikarum","Klassrum"]
    for row in data:
        predictions=predictioncounter(row,trees,labels)
        high=max(predictions)
        print("Ratio:",high,"/",len(trees))
        

In [828]:
randomforest(forrest,testing_data)

Actual: Klassrum. Predicted: {'Klassrum': '100%'}
Actual: Klassrum. Predicted: {'Klassrum': '100%'}
Actual: Klassrum. Predicted: {'Klassrum': '100%'}
Actual: Klassrum. Predicted: {'Klassrum': '100%'}
Actual: Klassrum. Predicted: {'Klassrum': '100%'}
Actual: Klassrum. Predicted: {'Klassrum': '100%'}
Actual: Klassrum. Predicted: {'Klassrum': '100%'}
Actual: Klassrum. Predicted: {'Klassrum': '100%'}
Ratio: 8 / 8
Actual: Fikarum. Predicted: {'Fikarum': '100%'}
Actual: Fikarum. Predicted: {'Klassrum': '100%'}
Actual: Fikarum. Predicted: {'Klassrum': '100%'}
Actual: Fikarum. Predicted: {'Fikarum': '100%'}
Actual: Fikarum. Predicted: {'Klassrum': '100%'}
Actual: Fikarum. Predicted: {'Klassrum': '100%'}
Actual: Fikarum. Predicted: {'Klassrum': '100%'}
Actual: Fikarum. Predicted: {'Fikarum': '100%'}
Ratio: 5 / 8
Actual: Fikarum. Predicted: {'Fikarum': '100%'}
Actual: Fikarum. Predicted: {'Klassrum': '100%'}
Actual: Fikarum. Predicted: {'Klassrum': '100%'}
Actual: Fikarum. Predicted: {'Fikarum':